## Data Preparation

### Parsing the annotated xml file for the image and create the YOLO compliant annotation

In [ ]:
# %pip install numpy pandas lxml opencv-python
import numpy as np
import pandas as pd
from glob import glob
import xml.etree.ElementTree as xet
import os
import cv2

In [3]:
df = pd.read_csv("./labels.csv")
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [15]:
def parsing(path):
  path = path.replace('\\', '/')
  parser = xet.parse(path).getroot()
  name = parser.find("filename").text
  filename = f"./images/{name}"

  parser_size = parser.find("size")
  width = int(parser_size.find("width").text)
  height =  int(parser_size.find("height").text)
  return filename, width, height

df[["filename","width","height"]] = df["filepath"].apply(parsing).apply(pd.Series)
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images\N1.xml,1093,1396,645,727,./images/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,./images\N100.xml,134,301,312,350,./images/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,./images\N101.xml,31,139,128,161,./images/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,./images\N102.xml,164,316,216,243,./images/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,./images\N103.xml,813,1067,665,724,./images/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


### Yolo conventions
center_x => X co-ordinate of the center of bounding box
center_y => Y co-ordinate of the center of bounding box
width => Width of the bounding box
height => Height of the bounding box

In [16]:
df["center_x"] = (df["xmax"] + df["xmin"])/(2*df["width"])
df["center_y"] = (df["ymax"] + df["ymin"])/(2*df["height"])

df["bb_width"] = (df["xmax"] - df["xmin"])/df['width']
df["bb_height"] = (df["ymax"] - df["ymin"])/df['height']

df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images\N1.xml,1093,1396,645,727,./images/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,./images\N100.xml,134,301,312,350,./images/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,./images\N101.xml,31,139,128,161,./images/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,./images\N102.xml,164,316,216,243,./images/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,./images\N103.xml,813,1067,665,724,./images/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


### Split the data to train and test

In [20]:
df.shape

(225, 12)

In [18]:
### split the data into train and test
df_train = df.iloc[:200]
df_test = df.iloc[200:]

#### 1. Split to train folder

In [17]:
from shutil import copy
train_folder = './data_images/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()


(225, 12)

#### 1. Split to test folder

In [ ]:
test_folder = './data_images/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()